In [1]:
import pandas as pd
import sqlite3
from jinja2 import Environment, FileSystemLoader

In [4]:
# configura o jinja
templateLoader = FileSystemLoader("../../models/templates/")
environment = Environment(loader=templateLoader)

In [5]:
# Arquivos de template do jinja
FILE_CUSTOMER = environment.get_template("customer_data.sql.j2")
FILE_PAYMENT = environment.get_template("payment_data.sql.j2")
FILE_MERGE_COSTUMER_PAYMENT = environment.get_template("customer_payment_merge.sql.j2")   

In [6]:
# cria um banco de dados no caminho especificado
conn = sqlite3.connect('../../data/DB/credit_risk')
cursor = conn.cursor()

In [7]:
#carrega template do jinja
query_customer = templateEnv.get_template(FILE_CUSTOMER).render()
# exibe a tabela de clientes
customer = pd.read_sql_query(query_customer, conn)
customer.head(3)

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
0,1,54982665,5,1245.5,3,77000.0,2,15,5,109,5,151300,244.948974
1,0,59004779,4,1277.0,1,113000.0,2,8,-1,100,3,341759,207.173840
2,0,58990862,7,1298.0,1,110000.0,2,11,-1,101,5,72001,1.000000


In [8]:
#carrega template do jinja
query_payment = templateEnv.get_template(FILE_PAYMENT).render()
# exibe a tabela de pagamentos
payment = pd.read_sql_query(query_payment, conn)
payment.head(3)

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,58987402,0,0,0,0,1,10,16500.0,04/12/2016,0.0,NaN,None
1,58995151,0,0,0,0,1,5,NaN,04/12/2016,588720.0,491100.0,None
2,58997200,0,0,0,0,2,5,NaN,04/12/2016,840000.0,700500.0,22/04/2016


In [9]:
#carrega template do jinja
query_merge_customer_payment = templateEnv.get_template(FILE_MERGE_COSTUMER_PAYMENT).render(id="id")
# exibe a tabela com merge de customer e payment
merge_customer_payment = pd.read_sql_query(query_merge_customer_payment, conn)
merge_customer_payment.head(3)

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,...,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
0,58987402,0,0,0,0,1,10,16500.0,04/12/2016,0.0,...,1241.0,3,62000.0,2,15,9,95,4,60023,1.000000
1,58995151,0,0,0,0,1,5,NaN,04/12/2016,588720.0,...,NaN,2,126000.0,2,11,9,115,4,450028,224.267697
2,58997200,0,0,0,0,2,5,NaN,04/12/2016,840000.0,...,1376.0,1,150000.0,2,11,-1,110,4,60000,219.248717


In [10]:
# fechar conexao
conn.close()

## 1.0 Analise da tabela de clientes

In [11]:
# colunas da tabela
customer.columns

Index(['label', 'id', 'fea_1', 'fea_2', 'fea_3', 'fea_4', 'fea_5', 'fea_6',
       'fea_7', 'fea_8', 'fea_9', 'fea_10', 'fea_11'],
      dtype='object')

In [12]:
# Analise preliminar de vazios e quantidade de observacoes
print(customer.shape)
customer.info()
print("\n *** Aparentemente nao tem null em nenhuma coluna")

(1125, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1125 non-null   int64  
 1   id      1125 non-null   int64  
 2   fea_1   1125 non-null   int64  
 3   fea_2   976 non-null    float64
 4   fea_3   1125 non-null   int64  
 5   fea_4   1125 non-null   float64
 6   fea_5   1125 non-null   int64  
 7   fea_6   1125 non-null   int64  
 8   fea_7   1125 non-null   int64  
 9   fea_8   1125 non-null   int64  
 10  fea_9   1125 non-null   int64  
 11  fea_10  1125 non-null   int64  
 12  fea_11  1125 non-null   float64
dtypes: float64(3), int64(10)
memory usage: 114.4 KB

 *** Aparentemente nao tem null em nenhuma coluna


In [13]:
# verifica os nulls
customer.isnull().sum()

label       0
id          0
fea_1       0
fea_2     149
fea_3       0
fea_4       0
fea_5       0
fea_6       0
fea_7       0
fea_8       0
fea_9       0
fea_10      0
fea_11      0
dtype: int64

In [14]:
# analise dos valores
customer.describe()

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
count,1125.000000,1.125000e+03,1125.000000,976.000000,1125.000000,1.125000e+03,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000
mean,0.200000,5.783677e+07,5.482667,1283.911373,2.333333,1.208836e+05,1.928889,10.872000,4.832889,100.802667,4.195556,164618.495111,134.999004
std,0.400178,1.817150e+06,1.383338,51.764022,0.878773,8.844523e+04,0.257125,2.676437,2.971182,11.988955,0.855679,152520.488281,112.616798
min,0.000000,5.498235e+07,1.000000,1116.500000,1.000000,1.500000e+04,1.000000,3.000000,-1.000000,64.000000,1.000000,60000.000000,1.000000
25%,0.000000,5.499050e+07,4.000000,1244.000000,1.000000,7.200000e+04,2.000000,8.000000,5.000000,90.000000,3.000000,60044.000000,1.000000
50%,0.000000,5.898975e+07,5.000000,1281.500000,3.000000,1.020000e+05,2.000000,11.000000,5.000000,105.000000,4.000000,72000.000000,173.205081
75%,0.000000,5.899799e+07,7.000000,1314.500000,3.000000,1.390000e+05,2.000000,11.000000,5.000000,111.000000,5.000000,151307.000000,202.484567
max,1.000000,5.900624e+07,7.000000,1481.000000,3.000000,1.200000e+06,2.000000,16.000000,10.000000,115.000000,5.000000,650070.000000,707.106781


In [15]:
# itens da coluna label
customer[['label']].value_counts()

label
0        900
1        225
dtype: int64

In [16]:
# analisando se a coluna id tem duplicatas
print(f'Qtd total de obs: {customer[["id"]].shape[0]}')
print(
    f'Qtd obs removendo as duplicatas: {customer[["id"]].drop_duplicates().shape[0]}'
)
print('\n*** nao tem duplicatas na coluna id')

Qtd total de obs: 1125
Qtd obs removendo as duplicatas: 1125

*** nao tem duplicatas na coluna id


In [17]:
# verifica se a coluna id segue um padrao
customer['len_id'] = customer['id'].astype('str').str.len()
print(customer['len_id'].value_counts())
print(' *** todos ids tem o mesmo comprimento')

8    1125
Name: len_id, dtype: int64
 *** todos ids tem o mesmo comprimento


In [18]:
# Verificando se existe vazios escondidos como espaco
customer.query(
    'label == " " or id == " " or fea_1 == " " or fea_2 == " " or fea_3 == " " or fea_4 == " "'
)

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11,len_id


In [19]:
print(' *** a base customer tem null nas colunas [fea_2]')

 *** a base customer tem null nas colunas [fea_2]


## 2.0 Analise da tabela de pagamento

In [20]:
# Analise preliminar de vazios e quantidade de observacoes
print(payment.shape)
payment.info()
print(
    "\n *** existem nulls nas colunas [prod_limit, update_date, highest_balance, report_date]"
)

(8250, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8250 entries, 0 to 8249
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               8250 non-null   int64  
 1   OVD_t1           8250 non-null   int64  
 2   OVD_t2           8250 non-null   int64  
 3   OVD_t3           8250 non-null   int64  
 4   OVD_sum          8250 non-null   int64  
 5   pay_normal       8250 non-null   int64  
 6   prod_code        8250 non-null   int64  
 7   prod_limit       2132 non-null   float64
 8   update_date      8224 non-null   object 
 9   new_balance      8250 non-null   float64
 10  highest_balance  7841 non-null   float64
 11  report_date      7136 non-null   object 
dtypes: float64(3), int64(7), object(2)
memory usage: 773.6+ KB

 *** existem nulls nas colunas [prod_limit, update_date, highest_balance, report_date]


In [21]:
# verifica nulls
payment.isna().sum()

id                    0
OVD_t1                0
OVD_t2                0
OVD_t3                0
OVD_sum               0
pay_normal            0
prod_code             0
prod_limit         6118
update_date          26
new_balance           0
highest_balance     409
report_date        1114
dtype: int64

In [22]:
# analise dos valores
payment.describe()

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,new_balance,highest_balance
count,8.250000e+03,8250.000000,8250.000000,8250.000000,8250.000000,8250.000000,8250.000000,2132.000000,8.250000e+03,7.841000e+03
mean,5.782173e+07,0.249091,0.127152,0.369212,187.681697,14.526667,8.232000,85789.702205,1.054042e+05,2.192027e+05
std,1.822724e+06,1.250197,0.860046,2.900320,1804.232570,12.053627,3.533055,74345.828184,1.887704e+06,2.814536e+06
min,5.498235e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.100000,-4.030320e+04,5.010000e+02
25%,5.499050e+07,0.000000,0.000000,0.000000,0.000000,4.000000,6.000000,37400.000000,0.000000e+00,2.345300e+04
50%,5.898905e+07,0.000000,0.000000,0.000000,0.000000,11.000000,10.000000,68200.000000,0.000000e+00,4.404700e+04
75%,5.899655e+07,0.000000,0.000000,0.000000,0.000000,25.000000,10.000000,112200.000000,2.494800e+04,1.005000e+05
max,5.900624e+07,34.000000,34.000000,35.000000,31500.000000,36.000000,27.000000,660000.000000,1.632120e+08,1.800005e+08


In [23]:
# analisando se a coluna id tem duplicatas
print(f'Qtd total de obs: {payment[["id"]].shape[0]}')
print(
    f'Qtd obs removendo as duplicatas: {payment[["id"]].drop_duplicates().shape[0]}'
)
print(
    '\n*** existem duplicatas o que faz sentido considerando que uma pessoa pode pagar mais de uma vez'
)

Qtd total de obs: 8250
Qtd obs removendo as duplicatas: 1125

*** existem duplicatas o que faz sentido considerando que uma pessoa pode pagar mais de uma vez


In [24]:
# verifica se a coluna id segue um padrao
payment['len_id'] = payment['id'].astype('str').str.len()
print(payment['len_id'].value_counts())
print(' *** todos ids tem o mesmo comprimento')

8    8250
Name: len_id, dtype: int64
 *** todos ids tem o mesmo comprimento


In [25]:
print(
    ' *** a base payment tem nulls nas colunas [prod_limit, update_date, highest_balance, report_date]'
)

 *** a base payment tem nulls nas colunas [prod_limit, update_date, highest_balance, report_date]


## 3.0 Analisando a sinergia entre as duas tabelas

In [26]:
# existem clientes da tabela de clientes que nao tem pagamento?
pagamento_clientes = (pd.merge(customer[['id', 'label']],
                               payment[['id', 'prod_code']],
                               how='left',
                               on=['id']))
print(f"tamanho da tabrla de merge: {pagamento_clientes.shape}")
print(pagamento_clientes.query("prod_code.isna()", engine="python").shape[0])
print("*** todos clientes da tabela de clientes tem algum pagamento")

tamanho da tabrla de merge: (8250, 3)
0
*** todos clientes da tabela de clientes tem algum pagamento


In [27]:
# testando o merge utilizando a query do template jinja
print(f"tamanho da tabrla de merge: {merge_customer_payment.shape}")
print(merge_customer_payment.query("prod_code.isna()", engine="python").shape[0])

tamanho da tabrla de merge: (8250, 25)
0
